In [1]:
#Import Dependencies
import os
import pandas as pd
import numpy as np
import cv2
import matplotlib.pyplot as plt
import time
import shutil
import warnings
import yaml
warnings.filterwarnings("ignore")
os.environ['KMP_DUPLICATE_LIB_OK']='True'

%matplotlib inline

start=time.time()

In [2]:
#Path of train and val information file
reef_path='/kaggle/input/reef-yaml/reef.yaml'

In [3]:
#Paths of trained models - Must be uploaded to kaggle 
model_path1='/kaggle/input/modelsl6/val2_l6.pt' #modelsbig/val2big.pt' #big 01, 12, 02 tested
model_path2='/kaggle/input/modelsl6/val1_l6.pt' #modelsbig/val1big.pt'
model_path='/kaggle/input/models/run1.pt' #run1,run2 - error with run3!!

In [4]:
#path of folder with downloaded data
base_path_data="/kaggle/input/tensorflow-great-barrier-reef/" 

In [6]:
#Download file arial.ttf from https://ultralytics.com/assets/Arial.ttf  needed for inference.
#More information in https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/297747

In [7]:
!mkdir -p /root/.config/Ultralytics 
!cp ../input/arialttf/Arial.ttf /root/.config/Ultralytics/

In [8]:
cd ..

/kaggle


In [9]:
cd input

/kaggle/input


In [10]:
%cd yolov5
%cd yolov5

/kaggle/input/yolov5
/kaggle/input/yolov5/yolov5


In [11]:
cd ..

/kaggle/input/yolov5


In [12]:
cd /kaggle/working

/kaggle/working


In [13]:
#To fix error ModuleNotFoundError: No module named 'greatbarrierreef.competition'
#Link is https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/291547
import sys
sys.path.append('../input/tensorflow-great-barrier-reef') 

In [14]:
# rm -rf /kaggle/working/test_images

In [15]:
#Create test_images folder
from PIL import Image
if not os.path.exists('/kaggle/working/test_images'):
    os.makedirs('/kaggle/working/test_images')

#To inspect images by saving them first

# import greatbarrierreef
# env = greatbarrierreef.make_env()   # initialize the environment
# iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission
# counting=0

# ar1=np.zeros((720,1280,3))
# ar2=np.zeros((720,1280,3))

# for (pixel_array, sample_prediction_df) in iter_test:
#     print(pixel_array.shape)
#     counting=counting+1
#     print(counting)
#     if counting==1:
#          ar1=pixel_array
#     elif counting==2:
#          ar2=pixel_array
#     im = Image.fromarray(pixel_array)
# #         idx=idx+1
#     im.save('/kaggle/working/test_images/'+str(counting)+".jpeg")
    
#     sample_prediction_df['annotations'] = '0.5 0 0 100 100'  # make your predictions here
#     env.predict(sample_prediction_df)   # register your predictions

In [16]:
cd ..

/kaggle


In [17]:
cd input

/kaggle/input


In [18]:
cd yolov5/yolov5

/kaggle/input/yolov5/yolov5


In [19]:
# Run inference on saved images

path="/kaggle/working/test_images/"
# path2="C:/Users/soyrl/Desktop/yolov5/data/reef.yaml"
# path3=path+'results/'
# !python detect.py --weights /kaggle/input/models/run1.pt /kaggle/input/modelsbig/val2big.pt /kaggle/input/modelsbig/val1big.pt --img 1280 --conf 0.01 --source $path --max-det 20 --save-txt --save-conf --augment --project $path --name results

# --device 0 
#--data $path2

# train_csv_path=base_path_data+"train.csv" #Set path to train.csv file
# train_csv=pd.read_csv(train_csv_path) #Load train_csv file to dataframe
# for file in os.listdir(path):
#     if 'results'!=file:
#         print(file)
#         series=train_csv[train_csv['video_frame']==int(file[:-4])]['video_id']==int(i)
#         print(train_csv.iloc[series[series].index[0]]) #.to_markdown()
#         print('\n')

In [20]:
cd /kaggle/working/test_images

/kaggle/working/test_images


In [21]:
#Problem when compare resulted images from each model compared to all combined!
#Seems like boxes are detected only from run1

# img=cv2.imread('/kaggle/working/test_images/results/3.jpeg') #Load image with most annotations #12679
# img=cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# plt.imshow(img)

In [22]:
#aaa

In [23]:
cd ..

/kaggle/working


In [25]:
#Functions to convert annotations to different formats and draw images and boxes

def voc2yolo(bboxes, image_height=720, image_width=1280):
    """
    voc  => [x1, y1, x2, y1]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]/ image_height
    
    w = bboxes[..., 2] - bboxes[..., 0]
    h = bboxes[..., 3] - bboxes[..., 1]
    
    bboxes[..., 0] = bboxes[..., 0] + w/2
    bboxes[..., 1] = bboxes[..., 1] + h/2
    bboxes[..., 2] = w
    bboxes[..., 3] = h
    
    return bboxes

def yolo2voc(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

def coco2yolo(bboxes, image_height=720, image_width=1280):
    """
    coco => [xmin, ymin, w, h]
    yolo => [xmid, ymid, w, h] (normalized)
    """
    
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # normolizinig
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]/ image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]/ image_height
    
    # converstion (xmin, ymin) => (xmid, ymid)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]/2
    
    return bboxes

def yolo2coco(bboxes, image_height=720, image_width=1280):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    # denormalizing
    bboxes[..., [0, 2]]= bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]]= bboxes[..., [1, 3]]* image_height
    
    # converstion (xmid, ymid) => (xmin, ymin) 
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    
    return bboxes

def voc2coco(bboxes, image_height=720, image_width=1280):
    bboxes  = voc2yolo(bboxes, image_height, image_width)
    bboxes  = yolo2coco(bboxes, image_height, image_width)
    return bboxes


def load_image(image_path):
    return cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)


def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    # Plots one bounding box on image img
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label:
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)

def draw_bboxes(img, bboxes, classes, class_ids, colors = None, show_classes = None, bbox_format = 'yolo', class_name = False, line_thickness = 2):  
     
    image = img.copy()
    show_classes = classes if show_classes is None else show_classes
    colors = (0, 255 ,0) if colors is None else colors
    
    if bbox_format == 'yolo':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:
            
                x1 = round(float(bbox[0])*image.shape[1])
                y1 = round(float(bbox[1])*image.shape[0])
                w  = round(float(bbox[2])*image.shape[1]/2) #w/2 
                h  = round(float(bbox[3])*image.shape[0]/2)

                voc_bbox = (x1-w, y1-h, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(get_label(cls)),
                             line_thickness = line_thickness)
            
    elif bbox_format == 'coco':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes:            
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                w  = int(round(bbox[2]))
                h  = int(round(bbox[3]))

                voc_bbox = (x1, y1, x1+w, y1+h)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)

    elif bbox_format == 'voc_pascal':
        
        for idx in range(len(bboxes)):  
            
            bbox  = bboxes[idx]
            cls   = classes[idx]
            cls_id = class_ids[idx]
            color = colors[cls_id] if type(colors) is list else colors
            
            if cls in show_classes: 
                x1 = int(round(bbox[0]))
                y1 = int(round(bbox[1]))
                x2 = int(round(bbox[2]))
                y2 = int(round(bbox[3]))
                voc_bbox = (x1, y1, x2, y2)
                plot_one_box(voc_bbox, 
                             image,
                             color = color,
                             label = cls if class_name else str(cls_id),
                             line_thickness = line_thickness)
    else:
        raise ValueError('wrong bbox format')

    return image

def get_bbox(annots):
    bboxes = [list(annot.values()) for annot in annots]
    return bboxes

def get_imgsize(row):
    row['width'], row['height'] = imagesize.get(row['image_path'])
    return row


np.random.seed(8)
colors = (np.random.randint(0, 255), np.random.randint(0, 255), np.random.randint(0, 255))
colors=(255,0,0)

In [26]:
#Functions to load model, make predictions, format them appropriately and show them

def load_model(Best_Model, conf=0.25, iou=0.50):
    import os
    print(os.listdir())
    model = torch.hub.load('.',#'/kaggle/input/barrie-reef-yolo5/yolov5', #./yolov5 #../input/yolov5/yolov5/
                           'custom',
                           path=Best_Model,
                           source='local',
                           force_reload=True)  # local repo
    model.conf = conf  # NMS confidence threshold
    model.iou  = iou  # NMS IoU threshold
    model.classes = None   # (optional list) filter by class, i.e. = [0, 15, 16] for persons, cats and dogs
    model.multi_label = False  # NMS multiple labels per box
    model.max_det = 1000  # maximum number of detections per image
#     print(model)
    return model


def predict(model, img, size=1280, augment=False): #size was 768
    height, width = img.shape[:2]
#     print(height)
    results = model(img, size=size, augment=augment)  # custom inference size
#     print(results)
#     print("above predict results")
    preds   = results.pandas().xyxy[0]
    bboxes  = preds[['xmin','ymin','xmax','ymax']].values
#     bboxes=np.round(bboxes)
#     print(bboxes)
    print("bboxes are {}".format(bboxes))

    if len(bboxes):
#         print(bboxes)
        bboxes  = voc2coco(bboxes,height,width).astype(int)
        confs   = preds.confidence.values
        print("Confidence values are {}".format(confs))

        return bboxes, confs
    else:
        return [],[]
    
def format_prediction(bboxes, confs):
    annot = ''
    if len(bboxes)>0:
        for idx in range(len(bboxes)):
            xmin, ymin, w, h = bboxes[idx]
            conf             = confs[idx]
            annot += f'{conf} {xmin} {ymin} {w} {h}'
            annot +=' '
        annot = annot.strip(' ')
    return annot

def show_img(img, bboxes, bbox_format='yolo'):
    names  = ['starfish']*len(bboxes)
    labels = [0]*len(bboxes)
    img    = draw_bboxes(img = img,
                           bboxes = bboxes, 
                           classes = names,
                           class_ids = labels,
                           class_name = True, 
                           colors = colors, 
                           bbox_format = bbox_format,
                           line_thickness = 2)
    return Image.fromarray(img).resize((800, 400))

In [27]:
cd ..

/kaggle


In [28]:
cd input/yolov5/yolov5

/kaggle/input/yolov5/yolov5


#### Inference on one model only - https://www.kaggle.com/ammarnassanalhajali/barrier-reef-yolov5-inference

#### Error fix taken from https://www.kaggle.com/questions-and-answers/70138

In [ ]:
# cd /kaggle/working/

In [29]:
# Best_Model = model_path

In [30]:
# import greatbarrierreef
# env = greatbarrierreef.make_env()# initialize the environment
# iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

# import torch
# from tqdm import tqdm
# from PIL import Image

# IMG_SIZE=1280

# CONF= 0.2
# IOU= 0.7

# model = load_model(Best_Model, conf=CONF, iou=IOU)

# only activate next for one model - taken from https://www.kaggle.com/questions-and-answers/70138

# for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
# #     print(img.shape)
#     print(pred_df)
    
#     bboxes, confs  = predict(model, img, size=IMG_SIZE, augment=True )
#     annot = format_prediction(bboxes, confs)
#     pred_df['annotations'] = annot
#     print("Final annotations are {}".format(annot))
#     env.predict(pred_df)
#     if idx<3:
#         display(show_img(img, bboxes, bbox_format='coco'))

#read only submission file error

#### Inference on an ensemble of models - Locally it can be done using detect.py as shown in https://github.com/ultralytics/yolov5/issues/318

In [31]:
#Taken from https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/291547
#to avoid error 'no module 'greatbarrierreef' below

# import sys
# sys.path.insert(0, base_path_data)

In [32]:
import greatbarrierreef
env = greatbarrierreef.make_env()# initialize the environment
iter_test = env.iter_test()      # an iterator which loops over the test set and sample submission

import torch
from tqdm import tqdm
from PIL import Image

IMG_SIZE=1280

CONF= 0.2 #0.2#was 0.15, also tried 0.05 and 0.25 and 0.3 gives error in scores!!
IOU= 0.7 #0.70 #was 0.5 made 0.3, 0.6, 0.9 last

#Each model can also have its own conf and iou!
model1 = load_model(model_path, conf=CONF, iou=IOU)  #conf=CONF1, iou=IOU1
model2 = load_model(model_path1, conf=CONF, iou=IOU)
model3 = load_model(model_path2, conf=CONF, iou=IOU)

['CONTRIBUTING.md', 'train.py', 'Dockerfile', '.pre-commit-config.yaml', 'utils', 'setup.cfg', 'LICENSE', 'tutorial.ipynb', 'hubconf.py', '.gitignore', 'README.md', 'export.py', 'models', '.dockerignore', 'requirements.txt', 'val.py', '.gitattributes', '.git', 'data', 'detect.py']


YOLOv5 🚀 v6.0-207-g8efe977 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)

Fusing layers... 
Model Summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 v6.0-207-g8efe977 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)



['CONTRIBUTING.md', 'train.py', 'Dockerfile', '.pre-commit-config.yaml', 'utils', 'setup.cfg', 'LICENSE', 'tutorial.ipynb', 'hubconf.py', '.gitignore', 'README.md', 'export.py', 'models', '.dockerignore', 'requirements.txt', 'val.py', '.gitattributes', '.git', 'data', 'detect.py']


Fusing layers... 
Model Summary: 476 layers, 76118664 parameters, 0 gradients
Adding AutoShape... 
YOLOv5 🚀 v6.0-207-g8efe977 torch 1.9.1 CUDA:0 (Tesla P100-PCIE-16GB, 16281MiB)



['CONTRIBUTING.md', 'train.py', 'Dockerfile', '.pre-commit-config.yaml', 'utils', 'setup.cfg', 'LICENSE', 'tutorial.ipynb', 'hubconf.py', '.gitignore', 'README.md', 'export.py', 'models', '.dockerignore', 'requirements.txt', 'val.py', '.gitattributes', '.git', 'data', 'detect.py']


Fusing layers... 
Model Summary: 476 layers, 76118664 parameters, 0 gradients
Adding AutoShape... 


In [33]:
cd ..

/kaggle/input/yolov5


In [34]:
cd ..

/kaggle/input


In [35]:
cd ..

/kaggle


In [36]:
cd working

/kaggle/working


#### Ensemble methods - taken from https://www.kaggle.com/ammarnassanalhajali/ensembling-boxes-nms-soft-nms-nmw-wbf-yolov5

In [37]:
#Taken from https://www.kaggle.com/vgarshin/ensemble-boxes-104
sys.path.append('../input/ensemble-boxes-104/ensemble_boxes-1.0.4')
from ensemble_boxes import *

In [38]:
def run_nms(bboxes, confs, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None): #3rd classs,
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = nms(boxes, scores, labels, weights=weights, iou_thr=iou_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_nmw(bboxes, confs, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None): #3rd classs
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = non_maximum_weighted(boxes, scores, labels, weights=weights, iou_thr=iou_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_soft_nms(bboxes, confs, image_size, iou_thr=0.50, skip_box_thr=0.0001,sigma=0.1, weights=None): #3rd classs
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = soft_nms(boxes, scores, labels, weights=weights, iou_thr=iou_thr, sigma=sigma, thresh=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

def run_wbf(bboxes, confs, image_size, iou_thr=0.50, skip_box_thr=0.0001, weights=None): #3rd classs
    boxes =  [bbox/(image_size-1) for bbox in bboxes]
    scores = [conf for conf in confs]    
    labels = [np.ones(conf.shape[0]) for conf in confs]
    boxes, scores, labels = weighted_boxes_fusion(boxes, scores, labels, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    boxes = boxes*(image_size-1)
    return boxes, scores, labels

In [39]:
Method ="WBF"# NMS,Soft-NMS,NMW or WBF
IMG_SIZE  = 1280 #3600 1280
AUGMENT   = True
iou_thr=IOU
skip_box_thr = 0.0001
sigma = 0.01 #only in soft-nms
weights = [1, 1, 1]#[1.5, 0.5 ,1.5] was also tested, [0.5,2,2], [0.5,1,1], [1,2,2]
#--------------------------------------------------------------------------
for idx, (img, pred_df) in enumerate(tqdm(iter_test)):
    print("pred df is {}".format(pred_df))
    bboxes1, scores1  = predict(model1, img, size=IMG_SIZE, augment=AUGMENT) #,bbclasses1
    bboxes2, scores2  = predict(model2, img, size=IMG_SIZE, augment=AUGMENT) #,bbclasses2
    bboxes3, scores3 = predict(model3, img, size=IMG_SIZE, augment=AUGMENT) #,bbclasses3 
    boxes_list=[]
    scores_list=[]
    labels_list=[]

    if len(bboxes1)>0:
        boxes_list.append(bboxes1)
        scores_list.append(scores1)
        bboxes1[:, 2] = bboxes1[:, 2] + bboxes1[:, 0]
        bboxes1[:, 3] = bboxes1[:, 3] + bboxes1[:, 1]
    if len(bboxes2)>0:
        boxes_list.append(bboxes2)
        scores_list.append(scores2)
        bboxes2[:, 2] = bboxes2[:, 2] + bboxes2[:, 0]
        bboxes2[:, 3] = bboxes2[:, 3] + bboxes2[:, 1]
    if len(bboxes3)>0:
        boxes_list.append(bboxes3)
        scores_list.append(scores3)
        bboxes3[:, 2] = bboxes3[:, 2] + bboxes3[:, 0]
        bboxes3[:, 3] = bboxes3[:, 3] + bboxes3[:, 1]

    if (len(bboxes1) + len(bboxes2) + len(bboxes3))>0:
        if Method =="NMS":
            boxes, scores, labels  = run_nms(boxes_list, scores_list, IMG_SIZE, iou_thr=iou_thr,weights=weights) #3rd labels_list,
        elif Method =="Soft-NMS":
            boxes, scores, labels  = run_soft_nms(boxes_list, scores_list,IMG_SIZE, iou_thr,skip_box_thr,sigma,weights=weights) #labels_list,
        elif Method =="NMW":
            boxes, scores, labels  = run_nmw(boxes_list, scores_list, IMG_SIZE, iou_thr, skip_box_thr,weights=weights) #labels_list,
        elif Method =="WBF":
            boxes, scores, labels  = run_wbf(boxes_list, scores_list,IMG_SIZE, iou_thr, skip_box_thr,weights=weights) #labels_list,
        else:
            Method ="Only First Model(No Ensembling)"
            boxes, scores, labels  = bboxes1,scores1,bbclasses1 
        boxes[:, 2] = boxes[:, 2] - boxes[:, 0]
        boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    else: boxes=[]
    annot = format_prediction(boxes, scores)
    pred_df['annotations'] = annot
    print("New pred_df is {}".format(pred_df))
    print("Final annotations are {}".format(annot))
    env.predict(pred_df)
    if idx<3:
        display(show_img(img, boxes, bbox_format='coco'))
    print("------------------------------  " + Method +"   ---------------------------------------")

0it [00:00, ?it/s]

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
pred df is    index       annotations
0      0  0.99 123 456 1 1
bboxes are [[     434.93      194.94      460.22       219.2]]
Confidence values are [    0.21476]
bboxes are []


1it [00:05,  5.03s/it]

bboxes are []
New pred_df is    index                                        annotations
0      0  0.2147560566663742 434.0 194.0 25.0 23.9999999...
------------------------------  WBF   ---------------------------------------
pred df is    index       annotations
0      1  0.99 123 456 1 1
bboxes are [[      198.5      144.16      247.53      177.09]]
Confidence values are [     0.2428]
bboxes are []


2it [00:05,  2.24s/it]

bboxes are []
New pred_df is    index                                        annotations
0      1  0.24280232191085815 198.0 144.0 49.0 31.999999...
------------------------------  WBF   ---------------------------------------
pred df is    index       annotations
0      2  0.99 123 456 1 1
bboxes are [[     229.39      307.52      264.36      333.47]]
Confidence values are [    0.57852]
bboxes are []


3it [00:05,  1.87s/it]

bboxes are []
New pred_df is    index                               annotations
0      2  0.5785226225852966 229.0 307.0 34.0 25.0
------------------------------  WBF   ---------------------------------------


Useful Sources:

1) A Balanced Splitting Algorithm: https://www.kaggle.com/tatamikenn/balanced-fold-splitting-algorithm?scriptVersionId=83363247

2) Full Train and Inference Guide https://www.kaggle.com/andradaolteanu/greatbarrierreef-yolo-full-guide-train-infer

3) Why we should prefer to do video splits - to avoid data leakage https://www.kaggle.com/c/tensorflow-great-barrier-reef/discussion/293723

4) Ensemble Models Explained https://medium.com/inspiredbrilliance/object-detection-through-ensemble-of-models-fed015bc1ee0 and GitHub Repo https://github.com/ZFTurbo/Weighted-Boxes-Fusion

5) Object Detection Explained https://towardsdatascience.com/why-is-object-detection-so-messy-6a91191579b8

6) Explanatory Data Analysis (EDA) https://www.kaggle.com/andradaolteanu/greatbarrierreef-full-guide-to-bboxaugmentation

7) Image Enhancement https://www.kaggle.com/soumya9977/learning-to-sea-underwater-img-enhancement-eda

8) Systematic Evaluation of F2 score https://www.kaggle.com/locbaop/systematic-evaluate-f2-yolov5

9) Tracking (Not Used) https://www.kaggle.com/shigengtian/yolov5-is-all-you-need-and-tracking
and explanation of what it is https://www.kaggle.com/kocha1/only-yolov5-tracking-lb-642/comments
and files used for tracking https://www.kaggle.com/parapapapam/norfair031py3

10) Run jupyter notebook in Peregrine (University Cluster) https://wiki.hpc.rug.nl/peregrine/examples/jupyter